# Artificial Neural Networks

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score,r2_score
from sklearn.model_selection import KFold

In [16]:
df = pd.read_csv('data/variables3.csv')

X = df.drop('MR',axis=1)
y = df['MR']

In [17]:
df

,alt_sht,vel_sht,pit_sht,alt_tgt,vel_tgt,sin(hdg_tgt),cos(hdg_tgt),sin(rgt_tgt),cos(rgt_tgt),MR
0,44421.970347,537.496000,-24.833196,34353.468266,541.824384,-0.967068,-0.254518,0.305089,0.952324,26.328125
1,44548.033860,538.187792,10.192052,32946.738582,583.690520,0.782715,0.622380,-0.164024,0.986456,20.390625
2,44352.491182,591.255827,-22.500498,27989.703128,590.506078,-0.982489,-0.186323,-0.292326,0.956319,13.359375
3,44426.946447,528.648879,-41.096294,41864.869016,518.200251,-0.828868,0.559444,-0.376352,0.926477,9.609375
4,44393.321784,591.514922,-20.926105,19894.116005,567.848255,-0.328344,-0.944558,0.814632,0.579978,16.953125
...,...,...,...,...,...,...,...,...,...,...
5815,44441.578213,598.613378,-6.756109,42934.233400,442.350927,-0.998991,0.044913,0.402013,0.915634,33.203125
5816,44678.316987,593.082438,6.748711,24093.609529,576.417176,0.491367,0.870953,-0.305427,0.952216,15.078125
5817,44897.188245,594.954717,-44.614118,36910.748896,556.797646,-0.930452,0.366415,0.679538,0.733640,12.578125
5818,44219.306776,543.233281,40.566989,44279.634058,593.559503,0.788823,0.614620,-0.741844,0.670573,27.734375


## Scaling and  K-fold Cross Validation

### Scaling

In [18]:
scaler = MinMaxScaler()

In [19]:
X = scaler.fit_transform(X)

###  K-fold Cross Validation

In [20]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

## Creating a Model

In [21]:
np.random.seed(10)

In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1

metrics_df = pd.DataFrame(columns=['EVS','R2','MAE','MSE','RMSE'])

for train, test in kfold.split(X, y):

  # Define the model architecture
    model = Sequential()
    model.add(Dense(9,activation='relu'))
    model.add(Dense(500,activation='relu'))
    model.add(Dense(250,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(25,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(1))
    
    # Compile the model
    model.compile(optimizer='adam',loss='mse')
    
    # Early Stop
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    
    # Generate a print
    print('------------------------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    model.fit(X[train], 
              y[train],
              epochs=1000,
              batch_size=1,
              validation_data=(X[test], y[test]), 
              callbacks=[early_stop])
    
    # make predictions for test data
    predictions = model.predict(X[test])
    
    # Evaluation on Test Data
    metrics = {'EVS' : explained_variance_score(y[test], predictions),
               'R2'  : r2_score(y[test], predictions),
               'MAE' : mean_absolute_error(y[test],predictions),
               'MSE' : mean_squared_error(y[test],predictions),
               'RMSE': np.sqrt(mean_squared_error(y[test],predictions))}

    metrics = pd.DataFrame(metrics, index=['fold {}'.format(fold_no)])                          
    
    metrics_df = pd.concat([metrics_df,metrics], ignore_index=False)
                               
    metrics = pd.DataFrame(columns=['EVS','R2','MAE','MSE','RMSE'])
    
    # Increase fold number
    fold_no += 1
    

------------------------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 46.4753 - val_loss: 14.1451
Epoch 2/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 18.7561 - val_loss: 11.5370
Epoch 3/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 16.2861 - val_loss: 12.5439
Epoch 4/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 10.9287 - val_loss: 10.7233
Epoch 5/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 10.3749 - val_loss: 7.7063
Epoch 6/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 8.7978 - val_loss: 7.2901
Epoch 7/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 8.8047 - val_loss: 5.9773
Epoch 8/1000
4656/4656 [==============================] - 8s 2ms/step - loss: 8.5245 - val_loss: 15.4036
Epoch 9/1000
4656/4656 [===================

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
metrics_df

## Calculate the Mean and Standard Deviation of the Maximum Range
print("Mean of MR: {:.3f}".format(y.mean()))
print("STD of MR: {:.3f}".format(y.std()))

## Predicting a Single New Sample

In [48]:
j = 700

In [49]:
single_sample = df.drop('MR',axis=1).iloc[j]

In [50]:
single_sample = scaler.transform(single_sample.values.reshape(-1, 9))

In [51]:
single_sample

array([[0.12522089, 0.02787258, 0.87193441, 0.031173  , 0.34594907,
        0.02811896, 0.66531289, 0.91274298, 0.39841101]])

In [52]:
predicted = model.predict(single_sample)
predicted[0][0]

0.9523082

In [53]:
real = df['MR'].iloc[j]
real

0.078125

In [54]:
(real-predicted)[0][0]

-0.8741832

## Ready Model for Deployment

### Saving Model

In [55]:
model.save("models/model3.h5")

In [56]:
import tensorflow as tf
model.layers[1].weights

[<tf.Variable 'sequential_2/dense_19/kernel:0' shape=(9, 500) dtype=float32, numpy=
 array([[ 0.9161012 , -0.08981959, -0.01819481, ..., -0.01716812,
         -0.28162777, -0.04081641],
        [-0.20571229, -0.19958942, -0.15611082, ..., -0.38791233,
          0.27529487, -0.0113493 ],
        [-0.9495167 , -0.0769145 , -0.08025127, ..., -0.00800568,
         -0.04797319, -0.02971826],
        ...,
        [ 0.14866439, -0.20049852, -0.3893773 , ...,  0.23266563,
          0.36964425, -0.01897763],
        [ 0.04790989, -0.18123464, -0.29524052, ..., -0.589157  ,
         -0.09238208, -0.0255378 ],
        [-0.64229834,  0.02261791,  0.00289192, ...,  0.74265736,
          1.1804429 , -0.02199275]], dtype=float32)>,
 <tf.Variable 'sequential_2/dense_19/bias:0' shape=(500,) dtype=float32, numpy=
 array([ 1.36119932e-01, -8.57219696e-02,  5.25414608e-02, -3.72593403e-02,
        -1.10155594e+00,  1.33359715e-01, -8.92129689e-02, -6.74826026e-01,
        -3.28277409e-01, -2.37149019e-02,

### Saving Scaler

In [57]:
import joblib
joblib.dump(scaler,'models/scaler3.pkl');